#### Embedding For Customer Model


In [1]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary

In [2]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [3]:
#primero hacer un embeddings

#primero hacer un modelo basico que prediga customer sales

#desp usar xgboost,catboost y LGBM con embeddings

#desp ensamblar los 3 modelos. los pones con los mejores hyperparametros y usas el voting classifier. desp podes hacer una busqeuda hyperparametros para los pesos

#asi tener el final.

In [4]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [5]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'Promo', 'SchoolHoliday']

In [6]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]

In [7]:
uniques.T

,uniques
Store,1115
DayOfWeek,7
Year,3
Month,12
Day,31
StateHoliday,4
CompetitionMonthsOpen,25
Promo2Weeks,26
StoreType,4
Assortment,3


In [8]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 2, 'Year': 2, 'Month': 2,
'Day': 10, 'StateHoliday': 2, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 25, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1,'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1}

In [9]:
for v in cat_vars:
    uniques_ = df[v].unique()
    uniques_.sort()
    print(v, cat_var_dict[v], len(uniques_), uniques_)
    print()

Store 50 1115 [   0    1    2 ... 1112 1113 1114]

DayOfWeek 2 7 [0 1 2 3 4 5 6]

Year 2 3 [0 1 2]

Month 2 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Day 10 31 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]

StateHoliday 2 4 [0 1 2 3]

CompetitionMonthsOpen 2 25 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]

Promo2Weeks 1 26 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]

StoreType 2 4 [0 1 2 3]

Assortment 3 3 [0 1 2]

PromoInterval 3 4 [0 1 2 3]

CompetitionOpenSinceYear 4 21 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]

Promo2SinceYear 4 8 [0 1 2 3 4 5 6 7]

State 6 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Week 25 52 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]

Events 4 22 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20

In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Embedding, Input, Flatten, Concatenate, Dense, BatchNormalization, Activation, LeakyReLU, Dropout
from tensorflow.keras.regularizers import l2

In [17]:
def get_cat_vars_model(cat_vars, uniques, cat_var_dict):
    cat_vars_embed_outs = []
    cat_var_inputs = []
    for cat_var in cat_vars:
        cat_var_in = Input(shape=(1,), name=f"{cat_var}_input")
        cat_var_inputs.append(cat_var_in)
        embed_out = Embedding(uniques[cat_var][0], cat_var_dict[cat_var], name=f'{cat_var}_Embed')(cat_var_in)
        flatten_out = Flatten(name=f"{cat_var}_flat")(embed_out)
        cat_vars_embed_outs.append(flatten_out)
    return cat_var_inputs, cat_vars_embed_outs

def get_cont_vars_input(contin_vars, dense_layer=False):
    cont_vars_inputs = []
    cont_vars_outputs = []
    for cont_var in contin_vars:
        cont_var_in = Input(shape=(1,), name=f"{cont_var}_input")
        cont_vars_inputs.append(cont_var_in)
        if dense_layer:
            cont_var_out = Dense(1, name=f"{cont_var}_input", activation = 'linear')(cont_var_in)
            cont_vars_outputs.append(cont_var_out)
        else:
            cont_vars_outputs.append(cont_var_in)
    return cont_vars_inputs, cont_vars_outputs

In [12]:
cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model(cat_vars, uniques, cat_var_dict)
cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars)

In [13]:
merged = Concatenate(name='All_Concatenate')(cat_vars_embed_outs + cont_vars_inputs)
Embedding = Model(cat_var_inputs + cont_vars_inputs, merged, name = 'Embedding')

In [14]:
Embedding.summary()

Model: "Embedding"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Store_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
DayOfWeek_input (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
Year_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Month_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________

#### Data Split and Embedding

In [15]:
#divide train and val
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

In [16]:
print('train size: ' , df_train.shape[0])
print('val size: ', df_val.shape[0])
print('test size: ', df_test.shape[0])

train size:  814150
val size:  30188
test size:  41088


In [17]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

y_customers_train = df_train['Customers']
y_customers_val = df_val['Customers']

y_sales_train = df_train['Sales']
y_sales_val = df_val['Sales']

In [18]:
X_train = Embedding.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
X_val = Embedding.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
X_test = Embedding.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

1284/1284 [==============================] - 3s 2ms/step


#### Model To Predict Customers

In [29]:
from sklearn.model_selection import cross_val_score
from hyperopt import hp, tpe
from hyperopt.fmin import fmin
from lightgbm import LGBMRegressor

In [20]:
def objective(params):
    params = {        
        'learning_rate': '{:.4f}'.format(params['learning_rate'])
    }
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **params
    )
    
    score = -cross_val_score(clf, X_train, y_customers_train).mean()
    return score

In [21]:
space = {    
    'learning_rate': hp.loguniform('learning_rate', -4, -2),    
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

100%|████████████████████████████████████████████| 10/10 [1:38:50<00:00, 593.07s/trial, best loss: -0.9471684581277897]


In [22]:
best

{'learning_rate': 0.12384386656571827}

In [25]:
model_customer = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        learning_rate = 0.12384386656571827)        

In [27]:
model_customer.fit(X_train,y_customers_train)

LGBMRegressor(learning_rate=0.12384386656571827, min_child_samples=5,
              n_estimators=4000)

In [28]:
model_customer.score(X_train,y_customers_train)

0.9879142422090029

In [29]:
model_customer.score(X_val,y_customers_val)

0.9642866708024102

In [36]:
Customers_test = model_customer.predict(X_test)

In [61]:
df_customer = pd.DataFrame(Customers_test)

In [63]:
df_customer.to_csv('Customer_Test.csv')

### Embedding For Final Models

In [3]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')
df_customer = pd.read_csv('Customer_Test.csv')

In [7]:
df_test['Customers'] = df_customer['0']    #importar customer test

In [9]:
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']

In [10]:
contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
   'Promo', 'SchoolHoliday','Customers']

In [11]:
uniques = DataFrameSummary(df[cat_vars]).summary().loc[['uniques']]

In [12]:
uniques.T

,uniques
Store,1115
DayOfWeek,7
Year,3
Month,12
Day,31
StateHoliday,4
CompetitionMonthsOpen,25
Promo2Weeks,26
StoreType,4
Assortment,3


In [13]:
cat_var_dict = {'Store': 50, 'DayOfWeek': 2, 'Year': 2, 'Month': 2,
'Day': 10, 'StateHoliday': 2, 'CompetitionMonthsOpen': 2,
'Promo2Weeks': 1, 'StoreType': 2, 'Assortment': 3, 'PromoInterval': 3,
'CompetitionOpenSinceYear': 4, 'Promo2SinceYear': 4, 'State': 6,
'Week': 25, 'Events': 4, 'Promo_fw': 1,
'Promo_bw': 1,'SchoolHoliday_fw': 1,
'SchoolHoliday_bw': 1}

In [14]:
for v in cat_vars:
    uniques_ = df[v].unique()
    uniques_.sort()
    print(v, cat_var_dict[v], len(uniques_), uniques_)
    print()

Store 50 1115 [   0    1    2 ... 1112 1113 1114]

DayOfWeek 2 7 [0 1 2 3 4 5 6]

Year 2 3 [0 1 2]

Month 2 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Day 10 31 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]

StateHoliday 2 4 [0 1 2 3]

CompetitionMonthsOpen 2 25 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]

Promo2Weeks 1 26 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25]

StoreType 2 4 [0 1 2 3]

Assortment 3 3 [0 1 2]

PromoInterval 3 4 [0 1 2 3]

CompetitionOpenSinceYear 4 21 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]

Promo2SinceYear 4 8 [0 1 2 3 4 5 6 7]

State 6 12 [ 0  1  2  3  4  5  6  7  8  9 10 11]

Week 25 52 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51]

Events 4 22 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20

In [18]:
cont_vars_inputs,  cont_vars_outs= get_cont_vars_input(contin_vars)

In [19]:
cat_var_inputs, cat_vars_embed_outs = get_cat_vars_model(cat_vars, uniques, cat_var_dict)

In [20]:
merged_final = Concatenate(name='All_Concatenate_Final')(cat_vars_embed_outs + cont_vars_inputs)
Embedding_Final = Model(cat_var_inputs + cont_vars_inputs, merged_final, name = 'Embedding_Final_Models')

In [21]:
Embedding_Final.summary()

Model: "Embedding_Final_Models"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Store_input (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
DayOfWeek_input (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
Year_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
Month_input (InputLayer)        [(None, 1)]          0                                            
_____________________________________________________________________________

In [22]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]

In [23]:
print('train size: ' , df_train.shape)
print('val size: ', df_val.shape)
print('test size: ', df_test.shape)

train size:  (814150, 82)
val size:  (30188, 82)
test size:  (41088, 82)


In [24]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

y_sales_train = df_train['Sales']
y_sales_val = df_val['Sales']

In [25]:
X_train = Embedding_Final.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
X_val = Embedding_Final.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
X_test = Embedding_Final.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

1284/1284 [==============================] - 3s 2ms/step


#### LGBM


In [36]:
def objective(params):
    params = {        
        'learning_rate': '{:.4f}'.format(params['learning_rate'])
    }
    
    clf = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
        **params
    )
    
    score = -cross_val_score(clf, X_train, y_sales_train).mean()
    return score

In [37]:
space = {    
    'learning_rate': hp.loguniform('learning_rate', -4, -2),    
}

best = fmin(fn=objective,
            space=space,
            verbose=2,
            algo=tpe.suggest,
            max_evals=10)

100%|████████████████████████████████████████████| 10/10 [1:46:39<00:00, 639.98s/trial, best loss: -0.9750225263752629]


In [38]:
best

{'learning_rate': 0.07027882793237038}

In [40]:
lgbm_boost_model = LGBMRegressor(
        n_estimators=4000,
        min_child_samples= 5, max_depth= -1,
          learning_rate = 0.07027882793237038)

In [42]:
lgbm_boost_model.fit(X_train,y_sales_train)

LGBMRegressor(learning_rate=0.07027882793237038, min_child_samples=5,
              n_estimators=4000)

In [43]:
lgbm_boost_model.score(X_train,y_sales_train)

0.9898920105216946

In [44]:
lgbm_boost_model.score(X_val,y_sales_val)

0.976911178141236

In [45]:
prediction = lgbm_boost_model.predict(X_test)

In [49]:
prediction.shape

(41088,)

### Submission


In [140]:
submission = pd.read_csv('submision_baseline.csv')

In [141]:
submission['Sales'] = prediction

In [142]:
submission.set_index('Id',inplace = True)

In [143]:
submission

,Sales
Id,
1,4411.614580
2,7103.838554
3,8934.099153
4,7348.322016
5,6991.227866
...,...
41084,3290.236886
41085,8253.474945
41086,5953.094339


In [60]:
submission.to_csv('Submission_main.csv')

In [67]:
#private_score = 0.12006
#public_acore = 0.12757

In [144]:
df_test = pd.read_feather('test_normalized_data.fth')

In [145]:
df_test["Sales"] = prediction

In [146]:
df_test = df_test[['Open','Sales']]

In [147]:
df_test.index += 1

In [148]:
df_test['Sales'][df_test.Open == 0] = 0 

In [149]:
submission['Sales'] = df_test['Sales']

In [151]:
submission.to_csv('Submission_main_0.csv')

In [156]:
#private_score = 0.12006
#public_acore = 0.12757
#lo mismo equis de